In [ ]:
from kaggle_secrets import UserSecretsClient

In [ ]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [ ]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git
    

In [ ]:
%cd /kaggle/working/llama-3-finetune

In [ ]:
!git checkout -b Update-Eval-Notebook

In [ ]:
!git pull origin Update-Eval-Notebook

In [ ]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [ ]:
from datasets import load_dataset
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch

In [ ]:
split_ratio = 0.2
seed = 42

In [ ]:
dataset_name = "CohereForAI/aya_dataset"
dataset_type = "aya"

In [ ]:
lang_multi = "Nepali"
_, aya_nepali_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

In [ ]:
aya_nepali_test

In [ ]:
lang_multi = "Hindi"
_, aya_hindi_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

In [ ]:
aya_hindi_test

In [ ]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"
_, alpaca_nepali_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

In [ ]:
alpaca_nepali_test

In [ ]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"
_, alpaca_hindi_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

In [ ]:
alpaca_hindi_test

In [ ]:
alpaca_prompt_hindi = """Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
alpaca_prompt_nepali = """Below is an instruction in nepali that describes a task, paired with an input also in nepali that provides further context. Write a response in nepali that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

def run_inference(dataset, prompt, is_alpaca_format=True):
    print("HERE TEST")
    for i, row in enumerate(dataset):
        if is_alpaca_format:
            instruction = row["instruction"]
            input_text = row["input"]
            prompt = prompt.format(instruction, input_text, "")
        else:
            inputs = row["inputs"]
            prompt = prompt.format(inputs, "")
        inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
        generated_text = tokenizer.batch_decode(outputs)[0]
        dataset[i]["output"] = generated_text
        print(generated_text)
    return dataset

In [ ]:
aya_prompt_nepali = """Below is an input also in nepali that is an instruction or context. Write a response to the instruction or add more to the context in nepali that appropriately completes the request.

### Input:
{}

### Response:
{}"""

In [ ]:
# alpaca_hindi_test_s = alpaca_hindi_test.select(range(10)) 
# run_inference(alpaca_hindi_test_s, alpaca_prompt_hindi, is_alpaca_format=True)
aya_nepali_test_s = aya_nepali_test.select(range(10)) 
aya_nepali_test_s = run_inference(aya_nepali_test_s, aya_prompt_nepali, is_alpaca_format=False)

In [1]:
aya_nepali_test_s.to_csv("aya_nepali_test_s.csv")

NameError: name 'aya_nepali_test_s' is not defined